In [1]:
from examples import phase_music, entanglement_music_1, entanglement_music_2, entanglement_music_3

pygame 2.1.3 (SDL 2.0.22, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


Example en entangled sounds

In [2]:
entanglement_music_1(1)

FileNotFoundError: [Errno 2] No such file or directory: 'saved_songs/pentatoniq-2023-02-28 01:36:13.txt'

In [3]:
entanglement_music_2(4)

In [3]:
entanglement_music_3(4)

Examples of phases

In [2]:
phase_music(4, 3.14159265 * 2)

phase_music(4, 3.14159265 * 2)



A simple song that mixes circuits

In [3]:
entanglement_music_2(4)
entanglement_music_3(4)
entanglement_music_2(4)
entanglement_music_3(4)
phase_music(4, 3.14159265 * 2)
phase_music(4, 3.14159265 * 2)
entanglement_music_2(4)
entanglement_music_3(4)
